* this Notebook

### Set SDC SPA
#### Theory
* read all imagefiles in category xxx missing SPA set
* extact template and param Source

In [6]:
import pywikibot
from pywikibot import pagegenerators as pg

def list_template_usage(site_obj, tmpl_name):
    """
    Takes Site object and template name and returns a generator.

    The function expects a Site object (pywikibot.Site()) and
    a template name (String). It creates a list of all
    pages using that template and returns them as a generator.
    It only returns pages in the 121-namespace (property-talk-pages).
    The generator will load 50 pages at a time for iteration.
    """
    name = "{}:{}".format(site.namespace(10), tmpl_name)
    tmpl_page = pywikibot.Page(site, name)
    ref_gen = pg.ReferringPageGenerator(tmpl_page, onlyTemplateInclusion=True)
    filter_gen = pg.NamespaceFilterPageGenerator(ref_gen, namespaces=[121])
    generator = site.preloadpages(filter_gen, pageprops=True)
    return generator

def gather_template_usage(generator, template_name, header=None):
    """
    Takes a generator and a template name and returns usage list.

    The function can also take a header (list of strings) that will be
    the headers of the table (Needs to be the same dimension as the table).
    The first column needs to be a link to property (It will be made into
    a link. In this example the second column is a list of links to Q-items.
    """
    tmpl_usage = []
    if header != None:
        tmpl_usage.append(header)

    for page in generator:
        page_str = page.get()
        tmpl_list = pywikibot.textlib.extract_templates_and_params(page_str)

        for tmpl in tmpl_list:
            if template_name in tmpl:
                page_title = page.title().split(":")[1]
                property_link = "{{{{P|{}}}}}".format(page_title)
                tmpl_usage.append([property_link, tmpl[1]["list"]])
    return tmpl_usage

def create_table_string(data, highlight=(True, False, False, False),
                        table_class='wikitable', style=''):
    """
    Takes a list and returns a wikitable.

    @param data: The list that is converted to a wikitable.
    @type data: List (Nested)
    @param highlight: Tuple of rows and columns that should be highlighted.
                      (first row, last row, left column, right column)
    @type highlight: Tuple
    @param table_class: A string containing the class description.
                        See wikitable help.
    @type table_class: String
    @param style: A string containing the style description.
                  See wikitable help.
    @type style: String
    """
    last_row = len(data) - 1
    last_cell = len(data[0]) - 1

    table = '{{| class="{}" style="{}"\n'.format(table_class, style)
    for key, row in enumerate(data):
        if key == 0 and highlight[0] or key == last_row and highlight[1]:
            row_string = '|-\n! ' + '\n! '.join(cell for cell in row)
        else:
            row_string = '|-'
            cells = ''
            for ckey, cell in enumerate(row):
                if ckey == 0 and highlight[2]:
                    cells += '\n! ' + cell
                elif ckey == last_cell and highlight[3]:
                    cells += '\n! ' + cell
                else:
                    cells += '\n| ' + cell
            row_string += cells

        table += row_string + '\n'
    table += '|}'
    return table


site = pywikibot.Site("wikidata", 'wikidata')

template_name = "Constraint:Units"
tmpl_gen = list_template_usage(site, template_name)

header = ["Property", "Constraint:Units"]
tmpl_usage = gather_template_usage(tmpl_gen, template_name, header)

table = create_table_string(tmpl_usage)

write_page = pywikibot.Page(site, "Wikidata:Database reports/List of properties with Constraint:Unit templates and their values")
write_page.save(summary="Updating Table")

AttributeError: module 'pywikibot' has no attribute 'family'